In [2]:
# You will need a text file (renamed to cred.py) containing your credentialsimport cred
import cred
EMAIL = cred.EMAIL
PASS = cred.PASS

In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time

In [ ]:
# Defining current date formats
today = time.localtime()
today = today.tm_year,today.tm_mon,today.tm_mday
today_flt = str(today[2])+'/'+str(today[1]).zfill(2)+'/'+str(today[0]).zfill(2)
today = str(today[0])+'-'+str(today[1]).zfill(2)+'-'+str(today[2]).zfill(2)

In [ ]:
# Initiate scraper and navigate to Aptem
driver = webdriver.Chrome('chrome_driver/chromedriver')
URL = ''
driver.get(URL)
time.sleep(5)

# Login
input_email = driver.find_element_by_xpath('//input[@name="userNameOrEmail"]')
input_email.send_keys(EMAIL)
input_password = driver.find_element_by_xpath('//input[@name="password"]')
input_password.send_keys(PASS)
sign_in = driver.find_element_by_xpath('//input[@value="Sign in"]')
sign_in.click()

## Creating a list of student's learning plan's URLs and Names

### Only need to run this once

In [ ]:
# WIP (not currently working)
# drop_down = driver.find_element_by_xpath('//input[@class="k-input"]')
# drop_down.click()
# # cohort_sel = driver.find_elements_by_xpath(f'//span[@class="k-in"][@class={COHORT}]')
# # cohort_sel
# cohort_sel = driver.find_elements_by_xpath('//span[@class="k-in"]')[1]
# cohort_sel.click()

In [ ]:
# Filter so that you are the case owner
drop_down = driver.find_element_by_xpath('//span[@class="k-input"]')
drop_down.click()
time.sleep(5)
owner_sel.click()
# TODO: Adjust the index to select your name
owner_sel = driver.find_element_by_xpath('//li[@class="k-item"][@data-offset-index=0]')
search = driver.find_element_by_xpath('//input[@value="Search"]')
search.click()

# Collecting the results of users result pages (as only 10 fit in a single page we need to click through)
link_list = []
next_b = [0]
pages = int(driver.find_element_by_xpath("//span[@class='k-pager-input k-label']").text.split(' ')[-1])
next_b = driver.find_element_by_xpath("//span[@class='k-icon k-i-arrow-60-right']")

# We iterate through each page and extract the learning plan and user name of all learners on those pages
for i in range(pages):
    link_elements = driver.find_elements_by_xpath("//td[@role='gridcell']//a[contains(@href, 'LearningProgramme?')]")
    user_elements = driver.find_elements_by_xpath("//td[@role='gridcell']//a[contains(@href, 'JobSeekerProfile')]")
    print(len(link_elements))
#     for i in len(link_elements):
    for i,href in enumerate(link_elements):
        values = [user_elements[i].text,href.get_attribute('href')]
        link_list.append(values) 
    next_b.click()
    time.sleep(5)

In [ ]:
# Saving progress
pd.DataFrame(link_list).to_csv('names_and_links.csv',index=False)

## Data collection (run every time)

In [ ]:
# Using control point
names_and_links = pd.read_csv('names_and_links.csv')

In [ ]:
%%time
data = []
# for lp_url in link_list:
for i,lp_url in names_and_links.iterrows():    
# Extracting URL and user_ref
    user = lp_url[0]
    print(i,user)
    driver.get(lp_url[1])
    time.sleep(20)
    # Extending results on page
    date_from = driver.find_element_by_xpath("//input[@data-bind='kendoDatePicker: dateFrom']")
    date_to = driver.find_element_by_xpath("//input[@data-bind='kendoDatePicker: dateTo']")
    search_link = driver.find_element_by_xpath("//a[@data-bind='click: applyFilters']")
    date_from.clear()
    date_from.send_keys('01/09/2020')
    date_to.clear()
    date_to.send_keys(today_flt)
    search_link.click()
    time.sleep(5)
    # Extracting components
    components = driver.find_elements_by_xpath("//div[@class='lp-accordion-control lp-component']")
    for comp in components:
        data.append([user] + comp.text.split('\n')[0:3])
driver.quit()

# Creating nice dataframes and files

In [ ]:
df = pd.DataFrame(data,columns=['learner','category','component','status'])
df.to_csv(f'output/source/source_data_{today}.csv',index=False) # Saving data in case something goes wrong

# Selecting components that learners need to complete
cat_filt = (df['category'].isin(['Online training','Digital learning']))
comp_filt = (df.component.str.contains('Personal Development|Unit'))
df_clean = df[cat_filt & comp_filt].copy()

# Encoding completion status with numbers that would be easy to tell apart with conditional formating in excel
df_clean['completed'] = (df_clean.status == 'Completed').astype(int)
df_clean['referred'] = (df_clean.status == 'Referred').astype(int)
df_clean['pending'] = (df_clean.status.isin(['Evidence submitted','1outstanding approvals Evidence submitted'])).astype(int)
df_clean['status_num'] = df_clean.pending + df_clean.referred*50 + df_clean.completed*100
df_clean.loc[df_clean['status_num']<1,'status_num'] = -100

# Creating a component by learner matrix
report = df_clean.pivot_table(index='component',columns='learner',values='status_num',aggfunc='sum').reset_index()
report = df_clean.loc[df_clean.learner==names_and_links['0'][0],['component']].merge(report,on='component')

# Adds dates (from manually encoded csv currently) to be able to calculate backlog duration
report['due_dates'] = pd.to_datetime(pd.read_csv('due_dates.csv').iloc[:len(report),0],dayfirst=True)
report['month_block'] = np.floor((pd.to_datetime(today) - report['due_dates']).dt.days/28).astype(int)
report = report[['due_dates','month_block']+list(report.columns[:-2])]

# Classifies learners according to backlog duration criteria
compl_lst = []
for i in range(3):
    compl_lst.append(len(report[(report['month_block']>i)]))

lid = pd.read_csv('learners_id.csv') # Currently imported from official Corndel's cohort tracking doc
lid.set_index('name',inplace=True)
lid['backlog_months'] = '1-'
lid.loc[((report == 100).sum()[3:] < compl_lst[0]),'backlog_months'] = '>1'
lid.loc[((report == 100).sum()[3:] < compl_lst[1]),'backlog_months'] = '>2'
lid.loc[((report == 100).sum()[3:] < compl_lst[2]),'backlog_months'] = '3+'

# Adds the backlog category as row to componentXlearner matrix
report = report[report.columns[:3]].merge(report.T[3:].merge(lid[['backlog_months']]
                                ,left_index=True,right_index=True).T
             ,left_index=True,right_index=True,how='right')

# Exports matrix
report.to_csv(f'output/reports/report_{today}.csv',index=False)

# Exports email list sorted by backlog duration category (ideal for email automation)
lid.sort_values('backlog_months',inplace=True)
lid.to_csv(f'output/outreach/learners_outreach_{today}.csv') #

### Some fun EDA and applications

In [ ]:
report.tail()

In [ ]:
df[(df.learner==names_and_links['0'][0])&(df.component.str.contains('121'))]

In [ ]:
# Listing last 121 or QR
df[(df.category=='Review')
#    &(df.component.str.contains('Q'))
  ].groupby('learner').last()